# 2因子モデル標準化無し

In [14]:
import pandas as pd
from sklearn.decomposition import FactorAnalysis
import numpy as np

# 体育のテストデータを作成 (サンプルデータ)
data = {
    "100m走": [13.2, 12.8, 14.5, 13.1, 12.9, 13.3, 14.2, 12.7, 13.0, 14.1],
    "ソフトボール投げ": [25, 30, 22, 24, 28, 26, 29, 23, 27, 31],
    "3000m走": [12.5, 11.8, 13.2, 12.1, 11.7, 12.3, 13.0, 11.9, 12.4, 13.1],
    "走り幅跳び": [4.5, 4.8, 4.3, 4.7, 4.9, 4.6, 4.4, 4.7, 4.8, 4.2],
    "垂直飛び": [40, 42, 38, 41, 43, 39, 40, 42, 41, 39]
}

df = pd.DataFrame(data)

# 因子分析 (2因子モデル)
factor_analysis = FactorAnalysis(n_components=2, random_state=42)
factor_scores = factor_analysis.fit_transform(df)
factor_loadings = factor_analysis.components_.T

# 因子負荷量をデータフレームに変換
factor_loadings_df = pd.DataFrame(
    factor_loadings, 
    index=df.columns, 
    columns=["因子1負荷量(a_1)", "因子2負荷量(a_2)"]
)

# 因子スコアをデータフレームに変換
factor_scores_df = pd.DataFrame(
    factor_scores, 
    columns=["生徒毎の因子1スコア(f_1)", "生徒毎の因子2スコア(f_2)"]
)


In [15]:
factor_loadings_df

,因子1負荷量(a_1),因子2負荷量(a_2)
100m走,0.581409,-0.024305
ソフトボール投げ,0.017784,-2.692125
3000m走,0.510906,-0.007858
走り幅跳び,-0.208018,0.003233
垂直飛び,-1.335104,-0.288720


* 因子1負荷量は持久力が正の方向に対して表現されている。一方脚力の瞬発力系は負の方向に表現されている
* 因子2負荷量は筋力系が負の方向に対して表現されている。
* 一方で、因子1負荷量の垂直飛びが1を超えていたり、因子2負荷量のソフトボール投げが1を超えていることから、因子数が不十分で適切に説明できていないか、標準化がされていないか、生データが悪い可能性が高い。

In [16]:
factor_scores_df

,生徒毎の因子1スコア(f_1),生徒毎の因子2スコア(f_2)
0,0.148229,0.518596
1,-1.087388,-1.121056
2,1.603550,1.457237
3,-0.528572,0.808703
4,-1.304022,-0.567134
5,-0.039079,0.309040
6,1.084757,-0.888431
7,-0.941836,1.089701
8,-0.309784,-0.157468
9,1.374146,-1.449188


In [17]:
# 因子負荷量を取得
factor_1_loadings = factor_loadings_df["因子1負荷量(a_1)"].values
print(factor_1_loadings)

# 生徒1の元データ (観測変数スコア)
student_1_data = df.iloc[0].values
print(student_1_data)

# 因子1スコアを計算 (誤差項が0と仮定した際の負荷量の加重平均)
factor_1_score_manual = np.dot(factor_1_loadings, student_1_data) # 行列の内積を計算。この場合だと1ベクトル同士の内積
print(factor_1_score_manual)


[ 0.5814089   0.01778442  0.51090647 -0.20801794 -1.33510367]
[13.2 25.  12.5  4.5 40. ]
-39.834688532547844


In [20]:
# 共通性と独自性の計算 (改善版)
commonality = []

# 各観測変数について共通性と独自性を計算
for i in range(len(factor_loadings_df)):
    # 因子負荷量の二乗和を計算（共通性）
    communality_value = factor_loadings_df.iloc[i, 0]**2 + factor_loadings_df.iloc[i, 1]**2
    # 独自性 = 1 - 共通性
    uniqueness_value = 1 - communality_value
    # 結果をリストに追加
    commonality.append((communality_value, uniqueness_value))

# 結果をデータフレームに変換して表示
commonality_df = pd.DataFrame(commonality, columns=["共通性", "独自性"], index=factor_loadings_df.index)
commonality_df



,共通性,独自性
100m走,0.338627,0.661373
ソフトボール投げ,7.247851,-6.247851
3000m走,0.261087,0.738913
走り幅跳び,0.043282,0.956718
垂直飛び,1.865861,-0.865861


# 4因子モデル

In [21]:
# 体育のテストデータを作成 (サンプルデータ)
data = {
    "100m走": [13.2, 12.8, 14.5, 13.1, 12.9, 13.3, 14.2, 12.7, 13.0, 14.1],
    "ソフトボール投げ": [25, 30, 22, 24, 28, 26, 29, 23, 27, 31],
    "3000m走": [12.5, 11.8, 13.2, 12.1, 11.7, 12.3, 13.0, 11.9, 12.4, 13.1],
    "走り幅跳び": [4.5, 4.8, 4.3, 4.7, 4.9, 4.6, 4.4, 4.7, 4.8, 4.2],
    "垂直飛び": [40, 42, 38, 41, 43, 39, 40, 42, 41, 39]
}

df = pd.DataFrame(data)

# 因子分析 (2因子モデル)
factor_analysis = FactorAnalysis(n_components=4, random_state=42)
factor_scores = factor_analysis.fit_transform(df)
factor_loadings = factor_analysis.components_.T

# 因子負荷量をデータフレームに変換
factor_loadings_df = pd.DataFrame(
    factor_loadings, 
    index=df.columns, 
    columns=["因子1負荷量(a_1)", "因子2負荷量(a_2)","因子3負荷量(a_3)","因子4負荷量(a_4)"]
)

# 因子スコアをデータフレームに変換
factor_scores_df = pd.DataFrame(
    factor_scores, 
    columns=["生徒毎の因子1スコア(f_1)", "生徒毎の因子2スコア(f_2)","生徒毎の因子3スコア(f_3)","生徒毎の因子4スコア(f_4)"]
)

In [22]:
factor_loadings_df

,因子1負荷量(a_1),因子2負荷量(a_2),因子3負荷量(a_3),因子4負荷量(a_4)
100m走,0.583926,0.029835,-0.082415,-0.005658
ソフトボール投げ,0.003492,2.692736,0.142875,0.018883
3000m走,0.509728,0.010802,-0.009423,0.008950
走り幅跳び,-0.208563,-0.004177,-0.022311,0.011941
垂直飛び,-1.343089,0.295309,-0.288267,-0.041847


In [23]:
factor_scores_df

,生徒毎の因子1スコア(f_1),生徒毎の因子2スコア(f_2),生徒毎の因子3スコア(f_3),生徒毎の因子4スコア(f_4)
0,0.130317,-0.533791,0.712779,-0.048924
1,-1.076667,1.107156,0.243342,-0.031901
2,1.629405,-1.436982,-0.639495,0.019618
3,-0.516813,-0.808770,-0.123813,-0.048525
4,-1.291231,0.591974,-0.949344,-0.078323
5,-0.001739,-0.362143,1.050477,0.162572
6,1.073726,0.925724,-0.850397,-0.004185
7,-0.949107,-1.079451,0.054486,-0.295193
8,-0.364828,0.145353,-0.116515,0.522939
9,1.366937,1.450930,0.618480,-0.198076


* 2因子モデルの場合
* 一方で、因子1負荷量の垂直飛びが1を超えていたり、因子2負荷量のソフトボール投げが1を超えていることから、因子数が不十分で適切に説明できていない可能性が高い。
* 
* 4因子モデルでも、因子負荷量が1を超える負荷量があるので、chatgptが作った生データがおかしいか、標準化されていないことが原因である可能性が高い。


# 2因子モデルで標準化されたデータで因子モデル構築

In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import FactorAnalysis
import pandas as pd

# 体育のテストデータを作成
data = {
    "100m走": [13.2, 12.8, 14.5, 13.1, 12.9, 13.3, 14.2, 12.7, 13.0, 14.1],
    "ソフトボール投げ": [25, 30, 22, 24, 28, 26, 29, 23, 27, 31],
    "3000m走": [12.5, 11.8, 13.2, 12.1, 11.7, 12.3, 13.0, 11.9, 12.4, 13.1],
    "走り幅跳び": [4.5, 4.8, 4.3, 4.7, 4.9, 4.6, 4.4, 4.7, 4.8, 4.2],
    "垂直飛び": [40, 42, 38, 41, 43, 39, 40, 42, 41, 39]
}

df = pd.DataFrame(data)

# データを標準化
scaler = StandardScaler()
df_standardized = scaler.fit_transform(df)

# 因子分析 (2因子モデル)
factor_analysis = FactorAnalysis(n_components=2, random_state=42)
factor_scores = factor_analysis.fit_transform(df_standardized)
factor_loadings = factor_analysis.components_.T

# 因子負荷量をデータフレームに変換
factor_loadings_df = pd.DataFrame(
    factor_loadings, 
    index=df.columns, 
    columns=["因子1負荷量", "因子2負荷量"]
)

# 因子スコアをデータフレームに変換
factor_scores_df = pd.DataFrame(
    factor_scores, 
    columns=["因子1スコア", "因子2スコア"]
)

display(factor_loadings_df)

print("==========")

display(factor_scores_df)

,因子1負荷量,因子2負荷量
100m走,0.949820,-0.125090
ソフトボール投げ,-0.007976,-0.522045
3000m走,0.982299,-0.053983
走り幅跳び,-0.940500,-0.008542
垂直飛び,-0.904443,-0.304746


,因子1スコア,因子2スコア
0,0.154509,0.616204
1,-1.089412,-0.341325
2,1.628486,0.529857
3,-0.516138,0.427803
4,-1.332577,-0.966871
5,0.037275,1.480949
6,1.041828,-1.280164
7,-0.958462,0.307304
8,-0.314143,-0.131721
9,1.348633,-0.642037


In [25]:
# 共通性と独自性の計算 (改善版)
commonality = []

# 各観測変数について共通性と独自性を計算
for i in range(len(factor_loadings_df)):
    # 因子負荷量の二乗和を計算（共通性）
    communality_value = factor_loadings_df.iloc[i, 0]**2 + factor_loadings_df.iloc[i, 1]**2
    # 独自性 = 1 - 共通性
    uniqueness_value = 1 - communality_value
    # 結果をリストに追加
    commonality.append((communality_value, uniqueness_value))

# 結果をデータフレームに変換して表示
commonality_df = pd.DataFrame(commonality, columns=["共通性", "独自性"], index=factor_loadings_df.index)
commonality_df

,共通性,独自性
100m走,0.917805,0.082195
ソフトボール投げ,0.272594,0.727406
3000m走,0.967825,0.032175
走り幅跳び,0.884613,0.115387
垂直飛び,0.910887,0.089113


* 標準化したことによって、共通性と独自性が1を超えることなく、かつの和（分散）が1になっていることを確認できた。
* この結果から、100m走、3000m走、垂直飛びは共通性が高い。一方ソフトボール投げが最も共通性が低い
* このことは、種目テストの位置づけと総合的に鑑みて前者3つは身体特性（瞬発力、持久力）を反映していると考察できる
* 後者のソフトボール投げは身体特性（瞬発力、持久力）では図れない独自性が高く、モノを遠くに飛ばす技術（腕の振り方や体重移動技術）が反映されていると考察できる